<a href="https://colab.research.google.com/github/D-kn/Adult-Income-Dataset/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importation des librairies nécessaires**

In [ ]:
import pandas as pd # Analyse, nettoyage et préparation des données
import numpy as np # Manipulation des matrices ou tableaux 
import seaborn as sns # Création de tracées statistiques
from matplotlib import pyplot as plt # Manipulation des données sous formes de graphiques
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Importation de la dataset et ajout des noms de chaque colonne**

In [ ]:
# Importation dataset
file_path = '/content/drive/MyDrive/TP_APPRENTISSAGE/dat/adult.xls'
fichier = pd.read_csv(file_path,
                  names=["age", "workclass", "fnlwgt", "education","education-num", "marital-status", 
                         "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
                         "hours-per-week", "native-country","classification"])
dataset = fichier.rename(columns={'education-num': 'education_num','marital-status': 'marital_status',
                                  'capital-gain': 'capital_gain', 'capital-loss': 'capital_loss',
                                  'hours-per-week': 'hours_per_week', 'native-country': 'native_country'})
dataset

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,classification
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


##**Taille de la dataset**

In [ ]:
dataset['native_country'].value_counts();

In [ ]:
dataset.shape

(32561, 15)

##**Liste des colonnes**

In [ ]:
dataset.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'classification'],
      dtype='object')

##**Affichage du types de chaque variables**

In [ ]:
dataset.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
classification    object
dtype: object

##**Vérification des données manqantes**

In [ ]:
dataset[dataset.isnull().any(axis=1)].count()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
classification    0
dtype: int64

##**Affichage des 5 premieres lignes**

In [ ]:
dataset.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,classification
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


##**Affichage des 5 dernieres lignes**

In [ ]:
dataset.tail()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,classification
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


##**Variables Normalisation**

In [ ]:
# Reduction du nombre de column de la variable native_country
dataset.native_country.value_counts();

In [ ]:
def change_name_country(name):
  if (name == ' United-States') | (name == 1 ):
    return "United-States"
  else:
    return "Others"

In [ ]:
dataset["native_country"] = dataset["native_country"].map(change_name_country)
dataset

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,classification
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Others,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [ ]:
dataset.occupation.value_counts();

In [ ]:
#Ajout des occupations Protective-serv, Priv-house-serv Armed-Forces sur la nouvelle variable Other
dataset['occupation'] = dataset.occupation.str.replace('Priv-house-serv', 'Other').str.replace('Armed-Forces', 'Other').str.replace('Protective-serv', 'Other').str.replace('?', 'Other')
dataset.occupation.value_counts();

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [ ]:
dataset.race.value_counts();

In [ ]:
#Ajout des amérindiens sur Other
dataset['race'] = dataset.race.str.replace('Other', 'Other').str.replace('Amer-Indian-Eskimo', 'Other')
dataset.race.value_counts()

 White                 27816
 Black                  3124
 Asian-Pac-Islander     1039
 Other                   582
Name: race, dtype: int64

In [ ]:
dataset.marital_status.value_counts()

 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: marital_status, dtype: int64

In [ ]:
#Ajout des statuts Married-spouse-absent Married-AF-spouse sur Other
dataset['marital_status'] = dataset.marital_status.str.replace('Married-spouse-absent', 'Other').str.replace('Married-AF-spouse', 'Other')
dataset.marital_status.value_counts()

 Married-civ-spouse    14976
 Never-married         10683
 Divorced               4443
 Separated              1025
 Widowed                 993
 Other                   441
Name: marital_status, dtype: int64

In [ ]:
dataset.workclass.value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64

In [ ]:
#Ajout des workclass Without-pay Never-worked sur Self-emp-inc
dataset['workclass'] = dataset.workclass.str.replace('Self-emp-inc', 'Self-emp-inc').str.replace('Without-pay', 'Self-emp-inc').str.replace('Never-worked', 'Self-emp-inc')
dataset.workclass.value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1137
 Federal-gov           960
Name: workclass, dtype: int64

# **DashBoard**

In [ ]:
dataset.to_csv('dataset');

In [ ]:
!pwd;

/content


In [ ]:
!pip install pywedge

     |████████████████████████████████| 76.1 MB 64 kB/s 


In [ ]:
import pywedge as pw

In [ ]:
viz = pw.Pywedge_Charts(dataset, c=None, y = 'classification')

/usr/local/lib/python3.7/dist-packages/pywedge/pywedge.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  self.X = self.train.drop(self.y,1)


In [ ]:
charts = viz.make_charts()

HTML(value='<h2>Pywedge Make_Charts </h2>')